In [1]:
# import necessary libraries
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from sklearn.model_selection import train_test_split

/nas/home/pchhikar/anaconda3/envs/drrnNew/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-02 12:45:02.508554: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-02 12:45:02.508892: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-02 12:45:02.508921: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries ment

KeyboardInterrupt: 

In [ ]:

import pandas as pd
import numpy as np
from transformers import T5Tokenizer
from transformers import T5Config, T5ForConditionalGeneration
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_scheduler
from tqdm.auto import tqdm
import glob, json

In [ ]:
import json
from collections import defaultdict

mapping = defaultdict(dict)

with open("/data/prateek/github/see-food/images_100k/layer1_small_updated.json", "r") as f:
    data1 = json.load(f)
    
with open("/data/prateek/github/see-food/images_100k/det_ingrs_small_updated.json", "r") as f:
    data2 = json.load(f)
    
for element in data1:
    id = element["id"]
    recipe = element["instructions"]
    title = element["title"]
    recipe_final = []
    for x in recipe:
        recipe_final.append(x["text"])
    mapping[id]["recipe"] = "Recipe: " + " ".join(recipe_final)
    mapping[id]["title"] = "Title: "+ title


for element in data2:
    id = element["id"]
    ingrd = element["ingredients"]
    ingrd_final = []
    for x in ingrd:
        ingrd_final.append(x["text"])
    mapping[id]["ingredients"] = "Ingredients: " + ", ".join(ingrd_final)
    
    

    

In [ ]:
# convert the recipe data to a pandas DataFrame
import pandas as pd
recipe_df = pd.DataFrame.from_dict(mapping, orient='index', columns=['recipe', 'ingredients', 'title']).reset_index()
recipe_df = recipe_df.rename(columns={'index': 'id'})
recipe_df = recipe_df.drop('id', axis=1)



In [ ]:
recipe_df['recipe'] = recipe_df['title'] + '\n' + recipe_df['recipe']

In [ ]:
recipe_df.head()

,recipe,ingredients,title
0,Title: Crunchy Onion Potato Bake\nRecipe: Preh...,"Ingredients: milk, water, butter, mashed potat...",Title: Crunchy Onion Potato Bake
1,Title: Kombu Tea Grilled Chicken Thigh\nRecipe...,"Ingredients: chicken thighs, tea, white pepper",Title: Kombu Tea Grilled Chicken Thigh
2,Title: Strawberry Rhubarb Dump Cake\nRecipe: P...,"Ingredients: fresh rhubarb, frozen rhubarb, gr...",Title: Strawberry Rhubarb Dump Cake
3,Title: Yogurt Parfaits\nRecipe: Layer all ingr...,"Ingredients: non - fat vanilla yogurt, strawbe...",Title: Yogurt Parfaits
4,Title: Fennel-Rubbed Pork Tenderloin with Roas...,"Ingredients: fennel seeds, pork tenderloin, fe...",Title: Fennel-Rubbed Pork Tenderloin with Roas...


In [ ]:
# assume df is your dataframe
n = len(recipe_df) # get the number of rows
split_idx = int(n * 0.9) # define the split index
recipe_df_train = recipe_df[:split_idx] # get the first part
recipe_df_test = recipe_df[split_idx:] # get the second part

In [ ]:
len(recipe_df), len(recipe_df_train), len(recipe_df_test)

(36277, 32649, 3628)

In [ ]:
# initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

/nas/home/pchhikar/anaconda3/envs/drrnNew/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
from transformers import T5Tokenizer
from transformers import T5Config, T5ForConditionalGeneration
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_scheduler
from tqdm.auto import tqdm
import glob, json

In [ ]:
config = T5Config(
    vocab_size = tokenizer.vocab_size,
    pad_token_id = tokenizer.pad_token_id,
    eos_token_id = tokenizer.eos_token_id,
    decoder_start_token_id = tokenizer.pad_token_id,
    d_model = 300
)

model = T5ForConditionalGeneration(config)

sum(p.numel() for p in model.parameters() if p.requires_grad) / 1_000_000

35.444912

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class SummaryDataset(Dataset):
    def __init__(
        self,
        data: pd.DataFrame = recipe_df,
        tokenizer: T5Tokenizer = tokenizer,
        text_max_token_len: int = 200,
        summary_max_token_len: int = 15
    ):
        self.tokenizer = tokenizer
        self.data = data
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        text = data_row['ingredients']

        text_encoding = tokenizer(
            text,
            max_length=self.text_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        summary_encoding = tokenizer(
            data_row['recipe'],
            max_length=self.summary_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        labels = summary_encoding['input_ids']
        labels[labels == tokenizer.pad_token_id] = -100

        return dict(
            input_ids=text_encoding['input_ids'].flatten(),
            attention_mask=text_encoding['attention_mask'].flatten(),
            labels=labels.flatten(),
            decoder_attention_mask=summary_encoding['attention_mask'].flatten()
        )

dataset = SummaryDataset(recipe_df, tokenizer)
dataloader = DataLoader(dataset, shuffle=True, batch_size=32)


In [ ]:

num_epochs = 100
num_training_steps = num_epochs * len(dataloader)
progress_bar = tqdm(range(num_training_steps))


minLoss = 10000000
optimizer = AdamW(model.parameters(), lr = 0.001)

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(device)
model.train()
for epoch in range(num_epochs):
    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
        logits = outputs.logits
        
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        
        optimizer.zero_grad()
        progress_bar.update()
    
    # torch.save({
    #         'epoch': epoch,
    #         'model_state_dict': model.state_dict(),
    #         'optimizer_state_dict': optimizer.state_dict(),
    #         'loss': loss,
    #         }, f't5_small.pth')

    if loss < minLoss:
        print("model saved with loss = ", loss)
        torch.save(model.state_dict(), "t5_recipe.pt")
        minLoss = loss
        
    print(f'epoch: {epoch + 1} -- loss: {loss}')

  0%|          | 0/56700 [00:11<?, ?it/s]]


cuda


  1%|          | 1134/113400 [03:56<6:06:10,  5.11it/s]

model saved with loss =  tensor(4.0549, device='cuda:0', grad_fn=<NllLossBackward0>)


  1%|          | 1135/113400 [03:56<8:39:09,  3.60it/s]

...

epoch: 18 -- loss: 3.697070360183716


 19%|█▉        | 21546/113400 [1:18:14<5:04:07,  5.03it/s] 

epoch: 19 -- loss: 3.8653509616851807


 19%|█▉        | 21757/113400 [1:18:59<6:09:47,  4.13it/s]

KeyboardInterrupt: 

In [ ]:
def summarizeText(text, model=model):
    text_encoding = tokenizer(
        text,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
    generated_ids = model.generate(
        input_ids=text_encoding['input_ids'].to(device),
        attention_mask=text_encoding['attention_mask'].to(device),
        max_length=513,
        num_beams=4,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )    

    preds = [
            tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for gen_id in generated_ids
    ]
    return "".join(preds)

In [ ]:
print(summarizeText('Ingredients: frozen chopped broccoli, cooked rice, shredded cheddar cheese, shredded cheddar cheese, eggs, butter, milk, onion, garlic powder, basil, oregano, salt and pepper'))

51

In [ ]:
print(recipe_df["ingredients"][8])
print(recipe_df["recipe"][8])

Ingredients: frozen chopped broccoli, cooked rice, shredded cheddar cheese, shredded cheddar cheese, eggs, butter, milk, onion, garlic powder, basil, oregano, salt and pepper
Title: Broccoli Rice Bake
Recipe: Mix all ingredients and pour into a one quart baking dish. Bake at 350 for 20 minutes. Top with the remaining 1/2 cup cheese and continue baking for 10 minutes.


In [ ]:
print(len(recipe_df))

36277
